In [1]:
# Parameters
EMO_INDEX = 0
MODEL_INDEX = 7
N = 100


In [2]:
import os
import ast
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.dates as mdates
import statsmodels.api as sm # 用于统计检验
%matplotlib inline
plt.rcParams["font.sans-serif"]=["WenQuanYi Micro Hei"] #设置字体
plt.rcParams["axes.unicode_minus"]=False #该语句解决图像中的“-”负号的乱码问题

## 参数选择

In [3]:
# # Parameters
# MODEL_INDEX = 2
# EMO_INDEX = 0
# N = 20

In [4]:
ORI_DATA_PATH = '/data/public/fintechlab/zdh/Individual-Stock-Analysis/B_Temporal_Clustering/data'
ROOT_PATH = '/data/public/fintechlab/zdh/Individual-Stock-Analysis/B_Result_Analysis'
CLUSTER_PATH = f'{ROOT_PATH}/data/Comparison_of_Emotional_models'

MODEL = ['Emo-Dict/DLUT', 'Emo-Dict/Bian', 'Emo-Dict/Jiang',
         'Machine-learning/LR', 'Machine-learning/RF', 'Machine-learning/SVM', 
         'Deep-learning/BERT', 'Deep-learning/Ours'][MODEL_INDEX]
Emotion_Data_PATH = f'{ORI_DATA_PATH}/Emotion_Data/{MODEL}'   # 情绪数据路径
Financial_Data_PATH = f'{ORI_DATA_PATH}/Financial_Data' # 金融数据路径

In [5]:
EMO_MARKET = ['Shanghai_Composite_Index', 'CSI_300_Index', 'Chinext_Index'][EMO_INDEX]
EMO_INDEX_MAP = {
    'Shanghai_Composite_Index': '上证综合情绪值',
    'CSI_300_Index': '沪深300情绪值',
    'Chinext_Index': '创业板情绪值'
}
EMO_NAME = EMO_INDEX_MAP[EMO_MARKET]

## 数据预处理

In [6]:
## 读取聚类结果
cluster_data = pd.read_csv(f'{CLUSTER_PATH}/{EMO_MARKET}/{MODEL}.csv', dtype={"Stkcd": str})
cluster_data.columns = ['股票编号', '聚类标签', '公司名称']

# 尝试抽样直到满足每个簇至少出现3次
for _ in range(100):
    cluster_data = cluster_data.sample(n=N, random_state=np.random.randint(0, 10000))
    label_counts = cluster_data['聚类标签'].value_counts()
    
    if all(label_counts >= 3):
        break
else:
    raise ValueError(f"在 {RETRY_LIMIT} 次尝试中无法找到满足每个簇至少3个样本的抽样结果，请减少 N 或检查数据分布。")

cluster_data.head()

,股票编号,聚类标签,公司名称
38,300003,0,乐普医疗
17,000999,1,华润三九
72,600011,0,华能国际
75,600025,1,华能水电
31,002415,0,海康威视


In [7]:
## 读取股吧个股的数据
all_data = []
file_list = [f for f in os.listdir(Emotion_Data_PATH) if f.endswith('.csv')]

if MODEL != 'Deep-learning/Ours':
    for file in file_list:
        file_path = os.path.join(Emotion_Data_PATH, file)
        df = pd.read_csv(file_path)
        stock_code = os.path.splitext(file)[0] # 获取股票编号（文件名去掉扩展名）
        
        # 提取每一行的日期和情绪值
        for _, row in df.iterrows():
            new_row = {
                '股票编号': stock_code,
                '日期': row['日期'],
                '上证综合情绪值': row['上证综合情绪值'],
                '沪深300情绪值': row['沪深300情绪值'],
                '创业板情绪值': row['创业板情绪值']
            }
            all_data.append(new_row)
    guba_data = pd.DataFrame(all_data)
else:
    for file in tqdm(file_list, desc="读取文件"):
        file_path = os.path.join(Emotion_Data_PATH, file)
        try:
            df = pd.read_csv(file_path, usecols=['日期', '高维情绪变量'])  # 只读需要的列
        except Exception as e:
            print(f"读取失败 {file}: {e}")
            continue
    
        stock_code = os.path.splitext(file)[0]
    
        df['股票编号'] = stock_code
        all_data.append(df)
    guba_data = pd.concat(all_data, ignore_index=True)

guba_data

读取文件:   0%|          | 0/183 [00:00<?, ?it/s]

读取文件:   1%|          | 1/183 [00:00<01:05,  2.78it/s]

读取文件:   1%|          | 2/183 [00:00<00:40,  4.46it/s]

读取文件:   2%|▏         | 3/183 [00:00<00:47,  3.81it/s]

读取文件:   2%|▏         | 4/183 [00:01<01:50,  1.62it/s]

...

读取文件:  98%|█████████▊| 180/183 [01:13<00:01,  2.88it/s]

读取文件:  99%|█████████▉| 182/183 [01:13<00:00,  3.48it/s]

读取文件: 100%|██████████| 183/183 [01:15<00:00,  1.64it/s]

读取文件: 100%|██████████| 183/183 [01:15<00:00,  2.43it/s]

,日期,高维情绪变量,股票编号
0,2024-12-27,"[-0.18730907142162323, -0.22363345324993134, 0...",601933
1,2024-11-18,"[-0.6446929574012756, 0.14099551737308502, 0.4...",601933
2,2024-11-17,"[-0.14928916096687317, -0.10993610322475433, 0...",601933
3,2024-11-17,"[-0.2892271876335144, 0.09153766185045242, 0.2...",601933
4,2024-11-17,"[-0.18511946499347687, 0.3309328258037567, 0.2...",601933
...,...,...,...
495326,2021-04-09,"[-0.14503593742847443, 0.25753334164619446, 0....",601919
495327,2021-04-09,"[-0.30359137058258057, 0.26559382677078247, 0....",601919
495328,2021-04-08,"[0.15271537005901337, -0.08832913637161255, 0....",601919
495329,2021-04-08,"[-0.004539322108030319, 0.3122449517250061, 0....",601919


In [8]:
## 读取股票回报率的数据
return_data = pd.read_csv(f'{Financial_Data_PATH}/日个股回报率.csv', dtype={'股票编号': str})
return_data

,股票编号,日期,交易量,收益率变化
0,000002,2021-06-01,60990961,-0.003745
1,000002,2021-06-02,85354506,0.006015
2,000002,2021-06-03,50594187,-0.003363
3,000002,2021-06-04,71422364,-0.012748
4,000002,2021-06-07,64745280,-0.014812
...,...,...,...,...
154877,688981,2024-11-20,58507495,-0.017071
154878,688981,2024-11-21,56197106,0.002358
154879,688981,2024-11-22,79240108,-0.050588
154880,688981,2024-11-25,76905909,-0.029402


In [9]:
## 进行左连接，以 guba_data 为主表
merged_data = pd.merge(guba_data, return_data[['股票编号', '日期', '交易量', '收益率变化']], 
                       on=['股票编号', '日期'], 
                       how='left')
merged_data = pd.merge(merged_data, cluster_data, on='股票编号', how='left')
merged_data = merged_data.dropna()

merged_data['日期'] = pd.to_datetime(merged_data['日期'])
merged_data = merged_data[(merged_data['日期'] > '2021-05-06') & (merged_data['日期'] < '2024-11-18')]
merged_data

,日期,高维情绪变量,股票编号,交易量,收益率变化,聚类标签,公司名称
3152,2024-11-15,"[-0.06300923973321915, -0.10365696251392365, 0...",601995,56218159.0,-0.045024,0.0,中金公司
3153,2024-11-15,"[-0.14638908207416534, 0.06092992052435875, 0....",601995,56218159.0,-0.045024,0.0,中金公司
3154,2024-11-15,"[-0.0021159015595912933, -0.19582228362560272,...",601995,56218159.0,-0.045024,0.0,中金公司
3155,2024-11-14,"[-0.28736764192581177, 0.2803494334220886, 0.1...",601995,84300779.0,-0.009377,0.0,中金公司
3156,2024-11-14,"[-0.2497139573097229, 0.151638925075531, 0.262...",601995,84300779.0,-0.009377,0.0,中金公司
...,...,...,...,...,...,...,...
482305,2021-06-24,"[-0.3934313952922821, -0.15708090364933014, 0....",002387,10557933.0,-0.011364,1.0,维信诺
482306,2021-06-21,"[-0.1166326180100441, -0.002910083159804344, 0...",002387,14304915.0,-0.008612,1.0,维信诺
482309,2021-06-17,"[-0.20766767859458923, -0.013704018667340279, ...",002387,28618827.0,0.037328,1.0,维信诺
482310,2021-06-17,"[-0.027151180431246758, 0.16335681080818176, 0...",002387,28618827.0,0.037328,1.0,维信诺


In [10]:
# 对情绪值列进行 Min-Max 标准化
def min_max_normalization(df, cols):
    for col in cols:
        min_val = df[col].min()
        max_val = df[col].max()
        df[col] = 2 * (df[col] - min_val) / (max_val - min_val) - 1
    return df

# 对每个股票编号的数据进行标准化和按日期汇总
def process_data(df):
    if MODEL != 'Deep-learning/Ours':
        df_processed = pd.DataFrame()  # 用于存储结果
        for stock_code, stock_data in df.groupby('股票编号'):
            # 对每个股票编号内的数据进行标准化
            stock_data = min_max_normalization(stock_data, ['上证综合情绪值', '沪深300情绪值', '创业板情绪值'])
            
            # 按日期汇总数据，同时保留股票编号
            stock_summary = stock_data.groupby('日期').agg({
                '股票编号': 'first',  # 保留股票编号（在同一日期内它是相同的，使用 'first'）
                '上证综合情绪值': 'mean',  # 上证综合情绪值按日期取均值
                '沪深300情绪值': 'mean',  # 沪深300情绪值按日期取均值
                '创业板情绪值': 'mean',  # 创业板情绪值按日期取均值
                '交易量': 'mean',  # 交易量按日期求和
                '收益率变化': 'mean',  # 收益率变化按日期取均值
                '聚类标签': 'first', # 保留聚类标签（在同一日期内它是相同的，使用 'first'）
                '公司名称': 'first', # 保留公司名称（在同一日期内它是相同的，使用 'first'）
            }).reset_index(drop=False)
            
            # 合并每个股票的汇总数据
            df_processed = pd.concat([df_processed, stock_summary], ignore_index=True)
            df_processed = df_processed.sort_values(by=['股票编号', '日期'], ascending=[True, True])
        return df_processed
    else:
        df_processed = []    
        # 确保高维情绪变量是 list 类型（如果还没转过）
        if isinstance(df['高维情绪变量'].iloc[0], str):
            df['高维情绪变量'] = df['高维情绪变量'].progress_apply(ast.literal_eval)
        # 对每个股票编号处理，加 tqdm 进度条
        for stock_code, stock_data in tqdm(df.groupby('股票编号'), desc="Processing stocks"):
            for date, group in stock_data.groupby('日期'):
                emotion_matrix = np.array(group['高维情绪变量'].tolist())  # (样本数, 维度)

                avg_emotion = emotion_matrix.mean(axis=0).round(3).tolist()

                summary_row = {
                    '股票编号': stock_code,
                    '日期': date,
                    '高维情绪变量': avg_emotion,
                    '交易量': group['交易量'].mean(),
                    '收益率变化': group['收益率变化'].mean(),
                    '聚类标签': group['聚类标签'].iloc[0], # 保留聚类标签
                    '公司名称': group['公司名称'].iloc[0], # 保留公司名称
                }
                df_processed.append(summary_row)


    return pd.DataFrame(df_processed)

final_data = process_data(merged_data)
final_data

  0%|          | 0/200233 [00:00<?, ?it/s]

  0%|          | 4/200233 [00:00<1:23:35, 39.93it/s]

  0%|          | 63/200233 [00:00<09:12, 362.13it/s]

  0%|          | 122/200233 [00:00<07:10, 465.13it/s]

  0%|          | 181/200233 [00:00<06:30, 512.84it/s]

...

Processing stocks:  96%|█████████▌| 96/100 [00:18<00:00,  6.82it/s]

Processing stocks:  97%|█████████▋| 97/100 [00:18<00:00,  6.93it/s]

Processing stocks:  99%|█████████▉| 99/100 [00:19<00:00,  7.17it/s]

Processing stocks: 100%|██████████| 100/100 [00:19<00:00,  5.22it/s]

,股票编号,日期,高维情绪变量,交易量,收益率变化,聚类标签,公司名称
0,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",60990961.0,-0.003745,2.0,万科A
1,000002,2021-06-08,"[-0.276, -0.035, 0.247, 0.326, 0.018, 0.244, -...",44676494.0,0.004626,2.0,万科A
2,000002,2021-06-10,"[-0.083, 0.072, 0.273, 0.26, -0.087, 0.276, -0...",53800776.0,-0.010035,2.0,万科A
3,000002,2021-06-11,"[-0.114, 0.163, 0.381, 0.325, -0.091, 0.088, -...",75853738.0,-0.014035,2.0,万科A
4,000002,2021-06-15,"[-0.169, 0.147, 0.442, 0.245, -0.009, 0.368, -...",89915501.0,-0.020957,2.0,万科A
...,...,...,...,...,...,...,...
46536,688027,2024-11-08,"[-0.256, 0.111, 0.464, 0.062, -0.251, 0.017, -...",4293539.0,0.030305,1.0,国盾量子
46537,688027,2024-11-12,"[-0.003, 0.185, 0.305, 0.533, 0.034, 0.336, -0...",3943615.0,-0.061404,1.0,国盾量子
46538,688027,2024-11-13,"[-0.296, -0.023, 0.302, 0.175, 0.083, 0.216, -...",3551408.0,-0.017570,1.0,国盾量子
46539,688027,2024-11-14,"[-0.344, -0.02, 0.309, 0.283, 0.03, 0.295, -0....",6098264.0,0.054033,1.0,国盾量子


In [11]:
# 统计检验 : 按照聚类标签分组，并计算均值和方差
df = final_data.copy()
if MODEL != 'Deep-learning/Ours':
    statistical_result = df.groupby('聚类标签').agg(
        情绪值均值=(EMO_NAME, 'mean'),
        情绪值方差=(EMO_NAME, 'var'),
        收益率变化均值=('收益率变化', 'mean'),
        收益率变化方差=('收益率变化', 'var')
    ).reset_index()
else:
    statistical_result = None
# 输出结果
statistical_result

## 情绪值与股价二维展示图

In [12]:
## 绘制不同簇情绪值随时间变化图
if MODEL != 'Deep-learning/Ours':
    # 按 '聚类标签' 和 '日期' 分组，计算平均情绪值
    df_grouped = final_data.groupby(['聚类标签', '日期'])[['上证综合情绪值', '沪深300情绪值', '创业板情绪值']].mean().reset_index()
    
    # 为每个聚类标签绘制折线
    i = 0
    plt.figure(figsize=(12, 12))
    for cluster in df_grouped['聚类标签'].unique():
        if i == 0:
            x = 'red'
        elif i == 1:
            x = 'blue'
        elif i == 2:
            x = 'green'
        df_cluster = df_grouped[df_grouped['聚类标签'] == cluster]
        plt.plot(df_cluster['日期'], df_cluster[EMO_NAME], label=f'{EMO_NAME} - 聚类 {cluster}', color=x)
        i += 1
        
    plt.xlabel('日期')
    plt.ylabel('平均情绪值')
    plt.title('情绪值趋势图')
    
    
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonthday=15, interval=3))  # 每三个月显示
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 格式化为年-月
    
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    
    # plt.savefig(f'个股分析/{market_address}/图片/情绪值趋势图', dpi=300)
    plt.show()


In [13]:
## 绘制不同簇收益率随时间变化图
if MODEL != 'Deep-learning/Ours':
    # 按 '聚类标签' 和 '日期' 分组，计算平均收益率
    return_grouped = final_data.groupby(['聚类标签', '日期'])['收益率变化'].mean().reset_index()
    
    # 为每个聚类标签绘制折线
    i = 0
    plt.figure(figsize=(12, 12))
    for cluster in df_grouped['聚类标签'].unique():
        if i == 0:
            x = 'red'
        elif i == 1:
            x = 'blue'
        elif i == 2:
            x = 'green'
        df_cluster = return_grouped[return_grouped['聚类标签'] == cluster]
        plt.plot(df_cluster['日期'], df_cluster['收益率变化'], label=f'收益率变化 - 聚类 {cluster}', color=x)
        i += 1
    
    plt.xlabel('日期')
    plt.ylabel('平均收益率')
    plt.title('收益率趋势图')
    
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonthday=15, interval=3))  # 每三个月显示
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 格式化为年-月
    
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    
    # 显示图表
    # plt.savefig(f'个股分析/{market_address}/图片/收益率趋势图', dpi=300)
    plt.show()

## 三维展示图，分别展示簇是0，1，2的代表图

In [14]:
### 聚类0示例 ###
if MODEL != 'Deep-learning/Ours':
    # 复制数据集，确保不会修改原始数据
    merged_data_cleaned = final_data.copy()
    merged_data_cleaned['日期'] = pd.to_datetime(merged_data_cleaned['日期'])
    
    # 选择聚类标签为 0 的数据，找到数据最多的股票编号
    cluster_0_data = merged_data_cleaned[merged_data_cleaned['聚类标签'] == 0]
    top_stock_code = cluster_0_data['股票编号'].value_counts().idxmax()
    top_stock_data = cluster_0_data[cluster_0_data['股票编号'] == top_stock_code]
    
    # 创建 3D 图
    fig = plt.figure(figsize=(50, 50))
    ax = fig.add_subplot(111, projection='3d')
    
    # 提取绘图数据
    x = top_stock_data[EMO_NAME]  # 情绪值作为 x 轴
    y = mdates.date2num(top_stock_data['日期'])  # 时间转换为数值类型
    z = top_stock_data['收益率变化']  # 收益率变化作为 z 轴
    
    # 绘制三维折线图
    ax.plot(x, y, z, label=f"股票 {top_stock_code}", marker='o', linestyle='-', alpha=0.7)
    
    # 设置轴标签
    ax.set_xlabel(EMO_NAME)
    ax.set_ylabel('时间')
    ax.set_zlabel('收益率变化')
    ax.set_title(f'股票 {top_stock_code}: {EMO_NAME} 与收益率变化的三维关系')
    
    # 设置 y 轴时间刻度（每半年一个刻度）
    ax.yaxis.set_major_locator(mdates.MonthLocator(bymonthday=15, interval=6))  # 每半年一个刻度
    ax.yaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 按年月显示
    
    # 旋转日期标签防止重叠
    fig.autofmt_xdate()
    
    # 调整布局并保存图像
    plt.legend()
    # plt.savefig(f'个股分析/{market_address}/图片/聚类0_{top_stock_code}.png', dpi=300, bbox_inches='tight')
    plt.show()


In [15]:
### 聚类1示例 ###
if MODEL != 'Deep-learning/Ours':
    # 复制数据集，确保不会修改原始数据
    merged_data_cleaned = final_data.copy()
    merged_data_cleaned['日期'] = pd.to_datetime(merged_data_cleaned['日期'])
    
    # 选择聚类标签为 0 的数据，找到数据最多的股票编号
    cluster_0_data = merged_data_cleaned[merged_data_cleaned['聚类标签'] == 1]
    top_stock_code = cluster_0_data['股票编号'].value_counts().idxmax()
    top_stock_data = cluster_0_data[cluster_0_data['股票编号'] == top_stock_code]
    
    
    # 创建 3D 图
    fig = plt.figure(figsize=(50, 50))
    ax = fig.add_subplot(111, projection='3d')
    
    # 提取绘图数据
    x = top_stock_data[EMO_NAME]  # 情绪值作为 x 轴
    y = mdates.date2num(top_stock_data['日期'])  # 时间转换为数值类型
    z = top_stock_data['收益率变化']  # 收益率变化作为 z 轴
    
    # 绘制三维折线图
    ax.plot(x, y, z, label=f"股票 {top_stock_code}", marker='o', linestyle='-', alpha=0.7)
    
    # 设置轴标签
    ax.set_xlabel(EMO_NAME)
    ax.set_ylabel('时间')
    ax.set_zlabel('收益率变化')
    ax.set_title(f'股票 {top_stock_code}: {EMO_NAME} 与收益率变化的三维关系')
    
    # 设置 y 轴时间刻度（每半年一个刻度）
    ax.yaxis.set_major_locator(mdates.MonthLocator(bymonthday=15, interval=6))  # 每半年一个刻度
    ax.yaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 按年月显示
    
    # 旋转日期标签防止重叠
    fig.autofmt_xdate()
    
    # 调整布局并保存图像
    plt.legend()
    # plt.savefig(f'个股分析/{market_address}/图片/聚类1_{top_stock_code}.png', dpi=300, bbox_inches='tight')
    plt.show()


In [16]:
### 聚类2示例 ###
if MODEL != 'Deep-learning/Ours':
    # 复制数据集，确保不会修改原始数据
    merged_data_cleaned = final_data.copy()
    merged_data_cleaned['日期'] = pd.to_datetime(merged_data_cleaned['日期'])
    
    # 选择聚类标签为 0 的数据，找到数据最多的股票编号
    cluster_0_data = merged_data_cleaned[merged_data_cleaned['聚类标签'] == 2]
    top_stock_code = cluster_0_data['股票编号'].value_counts().idxmax()
    top_stock_data = cluster_0_data[cluster_0_data['股票编号'] == top_stock_code]
    
    
    # 创建 3D 图
    fig = plt.figure(figsize=(50, 50))
    ax = fig.add_subplot(111, projection='3d')
    
    # 提取绘图数据
    x = top_stock_data[EMO_NAME]  # 情绪值作为 x 轴
    y = mdates.date2num(top_stock_data['日期'])  # 时间转换为数值类型
    z = top_stock_data['收益率变化']  # 收益率变化作为 z 轴
    
    # 绘制三维折线图
    ax.plot(x, y, z, label=f"股票 {top_stock_code}", marker='o', linestyle='-', alpha=0.7)
    
    # 设置轴标签
    ax.set_xlabel(EMO_NAME)
    ax.set_ylabel('时间')
    ax.set_zlabel('收益率变化')
    ax.set_title(f'股票 {top_stock_code}: {EMO_NAME} 与收益率变化的三维关系')
    
    # 设置 y 轴时间刻度（每半年一个刻度）
    ax.yaxis.set_major_locator(mdates.MonthLocator(bymonthday=15, interval=6))  # 每半年一个刻度
    ax.yaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 按年月显示
    
    # 旋转日期标签防止重叠
    fig.autofmt_xdate()
    
    # 调整布局并保存图像
    plt.legend()
    # plt.savefig(f'个股分析/{market_address}/图片/聚类2_{top_stock_code}.png', dpi=300, bbox_inches='tight')
    plt.show()


## 进行聚类间实证分析

#### 读取个股金融数据

In [17]:
# 读取原始数据
microfinancial_data = pd.read_csv(f'{Financial_Data_PATH}/个股微观金融数据.csv', encoding='utf-8')

# 选择重要的列，日期和股票代码一定要包含
columns_to_select = [
    '日期_Date',  # 日期
    '股票代码_Stkcd',  # 股票代码
    '日振幅(%)_Dampltd',  # 日振幅
    '日收益率_Dret',  # 日收益率
    '市盈率_PE',  # 市盈率
    '成交量_Trdvol',  # 成交量
    '流通股日换手率(%)_DTrdTurnR',
    '总市值加权平均日资本收益_Daretmc',
]

# 提取相关列
microfinancial_data = microfinancial_data[columns_to_select]
microfinancial_data['股票代码_Stkcd'] = microfinancial_data['股票代码_Stkcd'].apply(lambda x: str(x).zfill(6))

# microfinancial_data = microfinancial_data[microfinancial_data['日期_Date'] == '2022-03-01'] #用于检验某一天数据
microfinancial_data

,日期_Date,股票代码_Stkcd,日振幅(%)_Dampltd,日收益率_Dret,市盈率_PE,成交量_Trdvol,流通股日换手率(%)_DTrdTurnR,总市值加权平均日资本收益_Daretmc
0,2023-02-01,000100,1.6667,0.0071,276.47,147751077.0,1.0357,0.0145
1,2023-02-02,000100,2.1277,-0.0095,273.86,188929965.0,1.3244,-0.0005
2,2023-02-03,000100,2.1480,-0.0048,272.55,200087013.0,1.4026,-0.0042
3,2023-02-06,000100,1.6787,-0.0072,270.59,134158688.0,0.9404,-0.0084
4,2023-02-07,000100,1.4493,0.0024,271.24,87200030.0,0.6113,0.0036
...,...,...,...,...,...,...,...,...
162617,2024-11-14,601566,3.0340,-0.0255,16.94,3223500.0,0.5610,-0.0146
162618,2024-11-15,601566,2.8643,0.0025,16.98,2824500.0,0.4915,-0.0112
162619,2024-11-18,601566,3.8509,-0.0025,16.94,3729993.0,0.6491,0.0009
162620,2024-11-19,601566,2.3661,0.0137,17.17,3367065.0,0.5859,0.0031


#### 加载之前的情绪聚类数据

In [18]:
final_data

,股票编号,日期,高维情绪变量,交易量,收益率变化,聚类标签,公司名称
0,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",60990961.0,-0.003745,2.0,万科A
1,000002,2021-06-08,"[-0.276, -0.035, 0.247, 0.326, 0.018, 0.244, -...",44676494.0,0.004626,2.0,万科A
2,000002,2021-06-10,"[-0.083, 0.072, 0.273, 0.26, -0.087, 0.276, -0...",53800776.0,-0.010035,2.0,万科A
3,000002,2021-06-11,"[-0.114, 0.163, 0.381, 0.325, -0.091, 0.088, -...",75853738.0,-0.014035,2.0,万科A
4,000002,2021-06-15,"[-0.169, 0.147, 0.442, 0.245, -0.009, 0.368, -...",89915501.0,-0.020957,2.0,万科A
...,...,...,...,...,...,...,...
46536,688027,2024-11-08,"[-0.256, 0.111, 0.464, 0.062, -0.251, 0.017, -...",4293539.0,0.030305,1.0,国盾量子
46537,688027,2024-11-12,"[-0.003, 0.185, 0.305, 0.533, 0.034, 0.336, -0...",3943615.0,-0.061404,1.0,国盾量子
46538,688027,2024-11-13,"[-0.296, -0.023, 0.302, 0.175, 0.083, 0.216, -...",3551408.0,-0.017570,1.0,国盾量子
46539,688027,2024-11-14,"[-0.344, -0.02, 0.309, 0.283, 0.03, 0.295, -0....",6098264.0,0.054033,1.0,国盾量子


#### 读取市场换手率与市盈率并进行插值

In [19]:
turnover_PE_rate = pd.read_csv(f'{Financial_Data_PATH}/市场整体换手率与市盈率.csv', encoding='utf-8')
turnover_PE_rate['统计日期'] = pd.to_datetime(turnover_PE_rate['统计日期'])

# 按照 '统计日期' 排序
turnover_PE_rate = turnover_PE_rate.sort_values(by='统计日期')

# 对 '平均市盈率' 和 '换手率(总股本)' 进行线性插值
turnover_PE_rate[['平均市盈率', '换手率(总股本)']] = turnover_PE_rate[['平均市盈率', '换手率(总股本)']].interpolate(method='linear', axis=0)

turnover_PE_rate = turnover_PE_rate.drop(columns=['统计口径编码'])
turnover_PE_rate.columns = ['统计日期', '市场平均市盈率', '市场换手率(总股本)']
turnover_PE_rate

,统计日期,市场平均市盈率,市场换手率(总股本)
0,2021-01-04,16.918000,1.140600
12,2021-01-04,25.986305,1.237217
11,2021-01-04,35.054611,1.333834
10,2021-01-04,44.122916,1.430451
8,2021-01-04,53.191221,1.527068
...,...,...,...
12364,2024-12-06,39.185061,2.176139
12363,2024-12-06,14.390000,1.292500
12374,2024-12-06,14.390000,1.292500
12368,2024-12-06,14.390000,1.292500


#### 数据拼接

In [20]:
# 将日期列转换为 datetime 格式
final_data['日期'] = pd.to_datetime(final_data['日期'])
microfinancial_data['日期_Date'] = pd.to_datetime(microfinancial_data['日期_Date'], errors='coerce')

# 使用 merge 进行内连接
result = pd.merge(
    final_data, 
    microfinancial_data, 
    left_on=['日期', '股票编号'],  # 左表的 key
    right_on=['日期_Date', '股票代码_Stkcd'], # 右表的 key
    how='inner'                   # 内连接
)

result = pd.merge(
    result, 
    turnover_PE_rate, 
    left_on=['日期'],  # 左表的 key
    right_on=['统计日期'], # 右表的 key
    how='inner'                   # 内连接
)

# 删除重复的列
result = result.drop(columns=['日期_Date', '股票代码_Stkcd', '交易量', '日收益率_Dret', '统计日期'])

# 修改列的位置，更加美观可视
columns_to_move = ['聚类标签', '公司名称']
result = pd.concat([result.drop(columns=columns_to_move), result[columns_to_move]], axis=1)

# 将成交量除以10000，并以万计数
result['成交量(百万)_Trdvol'] = result['成交量_Trdvol'] / 1_000_000
result = result.drop(columns=['成交量_Trdvol'])

cols_to_check = ['收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
                 '成交量(百万)_Trdvol', '流通股日换手率(%)_DTrdTurnR']

# 删除含有空值的行
result = result.dropna(subset=cols_to_check)
result

,股票编号,日期,高维情绪变量,收益率变化,日振幅(%)_Dampltd,市盈率_PE,流通股日换手率(%)_DTrdTurnR,总市值加权平均日资本收益_Daretmc,市场平均市盈率,市场换手率(总股本),聚类标签,公司名称,成交量(百万)_Trdvol
0,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",-0.003745,1.8727,7.44,0.6272,0.0039,24.523860,1.030223,2.0,万科A,60.990961
1,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",-0.003745,1.8727,7.44,0.6272,0.0039,31.761720,1.132746,2.0,万科A,60.990961
2,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",-0.003745,1.8727,7.44,0.6272,0.0039,38.999580,1.235269,2.0,万科A,60.990961
3,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",-0.003745,1.8727,7.44,0.6272,0.0039,46.237440,1.337792,2.0,万科A,60.990961
4,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",-0.003745,1.8727,7.44,0.6272,0.0039,53.475301,1.440316,2.0,万科A,60.990961
...,...,...,...,...,...,...,...,...,...,...,...,...,...
622786,603986,2024-11-15,"[-0.247, 0.242, 0.518, 0.274, -0.104, 0.222, -...",-0.028519,5.5086,157.60,3.1891,-0.0112,65.934994,2.786785,2.0,兆易创新,21.198816
622787,603986,2024-11-15,"[-0.247, 0.242, 0.518, 0.274, -0.104, 0.222, -...",-0.028519,5.5086,157.60,3.1891,-0.0112,48.663329,2.336790,2.0,兆易创新,21.198816
622788,603986,2024-11-15,"[-0.247, 0.242, 0.518, 0.274, -0.104, 0.222, -...",-0.028519,5.5086,157.60,3.1891,-0.0112,31.391665,1.886795,2.0,兆易创新,21.198816
622789,603986,2024-11-15,"[-0.247, 0.242, 0.518, 0.274, -0.104, 0.222, -...",-0.028519,5.5086,157.60,3.1891,-0.0112,14.120000,1.436800,2.0,兆易创新,21.198816


In [21]:
# 查看匹配的公司数
unique_stock_count = result['股票编号'].nunique()
print(unique_stock_count)

99


#### 回归检验

In [22]:
## 单因子：总市值加权平均日资本收益回归检验
df = result.copy()

# 自变量是总市值加权平均日资本收益
X = df[['总市值加权平均日资本收益_Daretmc']]

# 要拟合的因变量列表
y_vars = [
    '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
    '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
]


# 添加常数项（截距项）
X_with_const = sm.add_constant(X)

# 创建一个空的列表存储回归结果
regression_results = []

# 对每个因变量进行回归分析
for y_var in y_vars:
    y = df[y_var]  # 当前的因变量

    # 对股票编号分组进行回归
    grouped = df.groupby('股票编号')
    
    r_squared_list = []
    
    for stock_code, group in grouped:
        X_group = group[['总市值加权平均日资本收益_Daretmc']]
        X_with_const_group = sm.add_constant(X_group)
        y_group = group[y_var]
        
        try:
            # 拟合回归模型
            model = sm.OLS(y_group, X_with_const_group)
            results = model.fit()

            # 提取回归R方
            r_squared_list.append(results.rsquared)
        except Exception as e:
            # 如果回归失败（如R方计算为-inf），跳过
            print(f"回归失败：股票编号 {stock_code}, 错误信息: {e}")
            r_squared_list.append(float('nan'))  # 将其R方设为NaN
    
    # 计算R方的平均值，忽略NaN值
    avg_r_squared = pd.Series(r_squared_list).mean()
    
    # 对于每个因变量，存储回归系数等信息
    model = sm.OLS(y, X_with_const)
    results = model.fit()

    regression_results.append({
        'Variable': y_var,
        'S': round(results.bse['总市值加权平均日资本收益_Daretmc'], 3),  # 标准误差
        'Econ': round(results.params['总市值加权平均日资本收益_Daretmc'], 3),  # 回归系数
        'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
        'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
        'Econ_t_value': round(results.tvalues['总市值加权平均日资本收益_Daretmc'], 3),  # Econ的t统计量，保留三位小数
        'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
    })

# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/单因子：总市值加权平均日资本收益.csv')
regression_df


,Variable,S,Econ,Constant,R_squared,Econ_t_value,Constant_t_value
0,收益率变化,0.003,1.077,0.001,22.686,427.446,18.875
1,日振幅(%)_Dampltd,0.230,11.761,3.403,0.417,51.069,1220.875
2,市盈率_PE,14.975,38.428,30.048,0.001,2.566,165.800
3,成交量(百万)_Trdvol,11.159,397.027,64.661,0.203,35.578,478.791
4,流通股日换手率(%)_DTrdTurnR,0.191,6.388,1.379,0.179,33.378,595.596


In [23]:
## 单因子：情绪回归检验
if MODEL != 'Deep-learning/Ours':
    df = result.copy()
    
    # 自变量是情绪值
    X = df[[EMO_NAME]]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    # 创建一个空的列表存储回归结果
    regression_results = []
    
    # 对每个因变量进行回归分析
    for y_var in y_vars:
        y = df[y_var]  # 当前的因变量
    
        # 对股票编号分组进行回归
        grouped = df.groupby('股票编号')
        
        for stock_code, group in grouped:
            X_group = group[[EMO_NAME]]
            X_with_const_group = sm.add_constant(X_group)
            y_group = group[y_var]
            
            # 拟合回归模型
            model = sm.OLS(y_group, X_with_const_group)
            results = model.fit()
    
            # 提取回归R方
            r_squared_list.append(results.rsquared)
        
        # 计算R方的平均值
        avg_r_squared = sum(r_squared_list) / len(r_squared_list)
        
        # 对于每个因变量，存储回归系数等信息
        model = sm.OLS(y, X_with_const)
        results = model.fit()
    
        regression_results.append({
            'Variable': y_var,
            'S': round(results.bse[EMO_NAME], 3),  # 标准误差，保留三位小数
            'Econ': round(results.params[EMO_NAME], 3),  # 回归系数，保留三位小数
            'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
            'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
            'Econ_t_value': round(results.tvalues[EMO_NAME], 3),  # Econ的t统计量，保留三位小数
            'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
        })
else:
    df = result.copy()
    
    # 拆分高维情绪变量为多个列（多变量）
    emo_array = np.array(df['高维情绪变量'].tolist())  # shape: (样本数, 维度)
    emo_df = pd.DataFrame(emo_array, columns=[f'emotion_{i}' for i in range(emo_array.shape[1])])
    
    # 合并回df
    df = pd.concat([df.reset_index(drop=True), emo_df], axis=1)
    
    # 构造多维自变量X
    EMO_COLUMNS = [f'emotion_{i}' for i in range(emo_array.shape[1])]
    X = df[EMO_COLUMNS]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    regression_results = []

    for y_var in y_vars:
        y = df[y_var]
        
        # 每个股票编号做回归，统计 R²
        grouped = df.groupby('股票编号')
        
        # 全部数据回归
        model = sm.OLS(y, X_with_const)
        results = model.fit()

        # 提取部分指标（示例提取第一个变量的参数）
        econ_param = results.params[EMO_COLUMNS[0]]
        econ_bse = results.bse[EMO_COLUMNS[0]]
        econ_t = results.tvalues[EMO_COLUMNS[0]]
        
        regression_results.append({
            'Variable': y_var,
            'S': round(econ_bse, 3),
            'Econ': round(econ_param, 3),
            'Constant': round(results.params['const'], 3),
            'R_squared_global': round(results.rsquared * 100, 3),
            'Econ_t_value': round(econ_t, 3),
            'Constant_t_value': round(results.tvalues['const'], 3)
        })
    


# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/单因子：{EMO_NAME}.csv')
regression_df


,Variable,S,Econ,Constant,R_squared_global,Econ_t_value,Constant_t_value
0,收益率变化,0.004,-0.029,0.027,9.792,-6.950,1.737
1,日振幅(%)_Dampltd,0.326,0.460,13.947,14.118,1.412,11.392
2,市盈率_PE,22.305,-175.035,187.896,4.566,-7.847,2.244
3,成交量(百万)_Trdvol,15.904,-1.150,135.795,12.798,-0.072,2.274
4,流通股日换手率(%)_DTrdTurnR,0.267,0.660,4.873,16.330,2.472,4.858


In [24]:
## 总体双因子回归检验
if MODEL != 'Deep-learning/Ours':
    df = result.copy()
    
    # 自变量是情绪值和总市值加权平均日资本收益
    X = df[[EMO_NAME, '总市值加权平均日资本收益_Daretmc']]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    # 创建一个空的列表存储回归结果
    regression_results = []
    
    # 对每个因变量进行回归分析
    for y_var in y_vars:
        y = df[y_var]  # 当前的因变量
    
        # 对股票编号分组进行回归
        grouped = df.groupby('股票编号')
   
        # 对于每个因变量，存储回归系数等信息
        model = sm.OLS(y, X_with_const)
        results = model.fit()
    
        regression_results.append({
            'Variable': y_var,
            'S': round(results.bse[EMO_NAME], 3),  # 标准误差，保留三位小数
            'Econ': round(results.params[EMO_NAME], 3),  # 回归系数，保留三位小数
            'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
            'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
            'Econ_t_value': round(results.tvalues[EMO_NAME], 3),  # Econ的t统计量，保留三位小数
            'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
        })

else:
    df = result.copy()
    
    # 拆分高维情绪变量为多个列（多变量）
    emo_array = np.array(df['高维情绪变量'].tolist())  # shape: (样本数, 维度)
    emo_df = pd.DataFrame(emo_array, columns=[f'emotion_{i}' for i in range(emo_array.shape[1])])
    
    # 合并回df
    df = pd.concat([df.reset_index(drop=True), emo_df], axis=1)
    
    # 构造多维自变量X
    EMO_COLUMNS = [f'emotion_{i}' for i in range(emo_array.shape[1])]
    X = df[['总市值加权平均日资本收益_Daretmc'] + EMO_COLUMNS]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    regression_results = []

    for y_var in y_vars:
        y = df[y_var]
        
        # 每个股票编号做回归，统计 R²
        grouped = df.groupby('股票编号')

        # 全部数据回归
        model = sm.OLS(y, X_with_const)
        results = model.fit()

        # 提取部分指标（示例提取第一个变量的参数）
        econ_param = results.params[EMO_COLUMNS[0]]
        econ_bse = results.bse[EMO_COLUMNS[0]]
        econ_t = results.tvalues[EMO_COLUMNS[0]]
        
        regression_results.append({
            'Variable': y_var,
            'S': round(econ_bse, 3),
            'Econ': round(econ_param, 3),
            'Constant': round(results.params['const'], 3),
            'R_squared': round(results.rsquared * 100, 3),
            'Econ_t_value': round(econ_t, 3),
            'Constant_t_value': round(results.tvalues['const'], 3)
        })
    

# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/双因子：总体分析.csv')
regression_df


,Variable,S,Econ,Constant,R_squared,Econ_t_value,Constant_t_value
0,收益率变化,0.004,-0.029,0.042,28.265,-7.794,3.003
1,日振幅(%)_Dampltd,0.326,0.460,14.096,14.409,1.414,11.532
2,市盈率_PE,22.305,-175.035,188.317,4.566,-7.847,2.249
3,成交量(百万)_Trdvol,15.892,-1.150,140.586,12.928,-0.072,2.356
4,流通股日换手率(%)_DTrdTurnR,0.267,0.660,4.948,16.437,2.474,4.936


In [25]:
## 聚类0的回归检验
if MODEL != 'Deep-learning/Ours':
    df = result[result['聚类标签'] == 0].copy()
    
    # 自变量是情绪值和总市值加权平均日资本收益
    X = df[[EMO_NAME, '总市值加权平均日资本收益_Daretmc']]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    # 创建一个空的列表存储回归结果
    regression_results = []
    
    # 对每个因变量进行回归分析
    for y_var in y_vars:
        y = df[y_var]  # 当前的因变量
    
        # 对股票编号分组进行回归
        grouped = df.groupby('股票编号')
        
        # 对于每个因变量，存储回归系数等信息
        model = sm.OLS(y, X_with_const)
        results = model.fit()
    
        regression_results.append({
            'Variable': y_var,
            'S': round(results.bse[EMO_NAME], 3),  # 标准误差，保留三位小数
            'Econ': round(results.params[EMO_NAME], 3),  # 回归系数，保留三位小数
            'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
            'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
            'Econ_t_value': round(results.tvalues[EMO_NAME], 3),  # Econ的t统计量，保留三位小数
            'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
        })

else:
    df = result[result['聚类标签'] == 0].copy()

    # 拆分高维情绪变量为多个列（多变量）
    emo_array = np.array(df['高维情绪变量'].tolist())  # shape: (样本数, 维度)
    emo_df = pd.DataFrame(emo_array, columns=[f'emotion_{i}' for i in range(emo_array.shape[1])])
    
    # 合并回df
    df = pd.concat([df.reset_index(drop=True), emo_df], axis=1)
    
    # 构造多维自变量X
    EMO_COLUMNS = [f'emotion_{i}' for i in range(emo_array.shape[1])]
    X = df[['总市值加权平均日资本收益_Daretmc'] + EMO_COLUMNS]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    regression_results = []

    for y_var in y_vars:
        y = df[y_var]
        
        # 每个股票编号做回归，统计 R²
        grouped = df.groupby('股票编号')
        
        # 全部数据回归
        model = sm.OLS(y, X_with_const)
        results = model.fit()

        # 提取部分指标（示例提取第一个变量的参数）
        econ_param = results.params[EMO_COLUMNS[0]]
        econ_bse = results.bse[EMO_COLUMNS[0]]
        econ_t = results.tvalues[EMO_COLUMNS[0]]
        
        regression_results.append({
            'Variable': y_var,
            'S': round(econ_bse, 3),
            'Econ': round(econ_param, 3),
            'Constant': round(results.params['const'], 3),
            'R_squared': round(results.rsquared * 100, 3),
            'Econ_t_value': round(econ_t, 3),
            'Constant_t_value': round(results.tvalues['const'], 3)
        })
    
# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/双因子：聚类0检验.csv')
regression_df


,Variable,S,Econ,Constant,R_squared,Econ_t_value,Constant_t_value
0,收益率变化,0.007,-0.004,0.153,31.228,-0.587,6.043
1,日振幅(%)_Dampltd,0.582,-0.708,16.436,18.831,-1.217,7.383
2,市盈率_PE,45.798,-125.558,311.692,9.797,-2.742,1.779
3,成交量(百万)_Trdvol,16.998,-16.243,421.669,28.047,-0.956,6.486
4,流通股日换手率(%)_DTrdTurnR,0.446,0.361,0.252,22.767,0.809,0.148


In [26]:
## 聚类1的回归检验
if MODEL != 'Deep-learning/Ours':
    df = result[result['聚类标签'] == 1].copy()
    
    # 自变量是情绪值和总市值加权平均日资本收益
    X = df[[EMO_NAME, '总市值加权平均日资本收益_Daretmc']]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    # 创建一个空的列表存储回归结果
    regression_results = []
    
    # 对每个因变量进行回归分析
    for y_var in y_vars:
        y = df[y_var]  # 当前的因变量
    
        # 对股票编号分组进行回归
        grouped = df.groupby('股票编号')

        # 对于每个因变量，存储回归系数等信息
        model = sm.OLS(y, X_with_const)
        results = model.fit()
    
        regression_results.append({
            'Variable': y_var,
            'S': round(results.bse[EMO_NAME], 3),  # 标准误差，保留三位小数
            'Econ': round(results.params[EMO_NAME], 3),  # 回归系数，保留三位小数
            'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
            'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
            'Econ_t_value': round(results.tvalues[EMO_NAME], 3),  # Econ的t统计量，保留三位小数
            'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
        })
else:
    df = result[result['聚类标签'] == 1].copy()
    
    # 拆分高维情绪变量为多个列（多变量）
    emo_array = np.array(df['高维情绪变量'].tolist())  # shape: (样本数, 维度)
    print('(样本数, 维度):', emo_array.shape)
    emo_df = pd.DataFrame(emo_array, columns=[f'emotion_{i}' for i in range(emo_array.shape[1])])
    
    # 合并回df
    df = pd.concat([df.reset_index(drop=True), emo_df], axis=1)
    
    # 构造多维自变量X
    EMO_COLUMNS = [f'emotion_{i}' for i in range(emo_array.shape[1])]
    X = df[['总市值加权平均日资本收益_Daretmc'] + EMO_COLUMNS]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    regression_results = []

    for y_var in y_vars:
        y = df[y_var]
        
        # 每个股票编号做回归，统计 R²
        grouped = df.groupby('股票编号')    

        # 全部数据回归
        model = sm.OLS(y, X_with_const)
        results = model.fit()

        # 提取部分指标（示例提取第一个变量的参数）
        econ_param = results.params[EMO_COLUMNS[0]]
        econ_bse = results.bse[EMO_COLUMNS[0]]
        econ_t = results.tvalues[EMO_COLUMNS[0]]
        
        regression_results.append({
            'Variable': y_var,
            'S': round(econ_bse, 3),
            'Econ': round(econ_param, 3),
            'Constant': round(results.params['const'], 3),
            'R_squared': round(results.rsquared * 100, 3),
            'Econ_t_value': round(econ_t, 3),
            'Constant_t_value': round(results.tvalues['const'], 3)
        })

# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/双因子：聚类1检验.csv')
regression_df


(样本数, 维度): (121563, 768)


,Variable,S,Econ,Constant,R_squared,Econ_t_value,Constant_t_value
0,收益率变化,0.009,-0.050,-0.015,27.418,-5.614,-0.424
1,日振幅(%)_Dampltd,0.787,-3.834,4.234,20.561,-4.870,1.372
2,市盈率_PE,61.980,-801.752,1049.277,13.856,-12.936,4.320
3,成交量(百万)_Trdvol,14.268,-27.316,-414.660,27.567,-1.914,-7.416
4,流通股日换手率(%)_DTrdTurnR,0.645,0.095,1.226,23.455,0.147,0.485


In [27]:
## 聚类2的回归检验
if MODEL != 'Deep-learning/Ours':
    df = result[result['聚类标签'] == 2].copy()
    
    # 自变量是情绪值和总市值加权平均日资本收益
    X = df[[EMO_NAME, '总市值加权平均日资本收益_Daretmc']]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    # 创建一个空的列表存储回归结果
    regression_results = []
    
    # 对每个因变量进行回归分析
    for y_var in y_vars:
        y = df[y_var]  # 当前的因变量
    
        # 对股票编号分组进行回归
        grouped = df.groupby('股票编号')
    
        # 对于每个因变量，存储回归系数等信息
        model = sm.OLS(y, X_with_const)
        results = model.fit()
    
        regression_results.append({
            'Variable': y_var,
            'S': round(results.bse[EMO_NAME], 3),  # 标准误差，保留三位小数
            'Econ': round(results.params[EMO_NAME], 3),  # 回归系数，保留三位小数
            'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
            'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
            'Econ_t_value': round(results.tvalues[EMO_NAME], 3),  # Econ的t统计量，保留三位小数
            'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
        })
else:
    df = result[result['聚类标签'] == 2].copy()
    
    # 拆分高维情绪变量为多个列（多变量）
    emo_array = np.array(df['高维情绪变量'].tolist())  # shape: (样本数, 维度)
    print('(样本数, 维度):', emo_array.shape)
    emo_df = pd.DataFrame(emo_array, columns=[f'emotion_{i}' for i in range(emo_array.shape[1])])
    
    # 合并回df
    df = pd.concat([df.reset_index(drop=True), emo_df], axis=1)
    
    # 构造多维自变量X
    EMO_COLUMNS = [f'emotion_{i}' for i in range(emo_array.shape[1])]
    X = df[['总市值加权平均日资本收益_Daretmc'] + EMO_COLUMNS]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    regression_results = []

    for y_var in y_vars:
        y = df[y_var]
        
        # 每个股票编号做回归，统计 R²
        grouped = df.groupby('股票编号')
        r_squared_list = []
        
        # 全部数据回归
        model = sm.OLS(y, X_with_const)
        results = model.fit()

        # 提取部分指标（示例提取第一个变量的参数）
        econ_param = results.params[EMO_COLUMNS[0]]
        econ_bse = results.bse[EMO_COLUMNS[0]]
        econ_t = results.tvalues[EMO_COLUMNS[0]]
        
        regression_results.append({
            'Variable': y_var,
            'S': round(econ_bse, 3),
            'Econ': round(econ_param, 3),
            'Constant': round(results.params['const'], 3),
            'R_squared': round(results.rsquared * 100, 3),
            'Econ_t_value': round(econ_t, 3),
            'Constant_t_value': round(results.tvalues['const'], 3)
        })

# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/双因子：聚类2检验.csv')
regression_df


(样本数, 维度): (286780, 768)


,Variable,S,Econ,Constant,R_squared,Econ_t_value,Constant_t_value
0,收益率变化,0.005,-0.030,-0.081,32.386,-5.854,-3.751
1,日振幅(%)_Dampltd,0.432,2.728,14.220,19.694,6.313,7.769
2,市盈率_PE,20.296,-36.807,212.170,11.869,-1.814,2.469
3,成交量(百万)_Trdvol,29.934,-4.252,320.203,16.779,-0.142,2.526
4,流通股日换手率(%)_DTrdTurnR,0.374,0.500,7.157,21.183,1.337,4.522
